<a href="https://colab.research.google.com/github/vickymlam/lab-5-vickymlam/blob/main/Copy_of_DATA_301_Lab_5_YOUR_NAMES_HERE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Authorship of the Federalist Papers

The _Federalist Papers_ were a set of 85 essays published between 1787 and 1788 to promote the ratification of the United States Constitution. They were originally published under the pseudonym "Publius". Although the identity of the authors was a closely guarded secret at the time, most of the papers have since been conclusively attributed to one of Hamilton, Jay, or Madison. The known authorships can be found in `https://dlsun.github.io/pods/data/federalist/authorship.csv`.

For 15 of the papers, however, the authorships remain disputed. (These papers can be identified from the `authorship.csv` file because the "Author" field is blank.) In this analysis, you will use the papers with known authorship to predict the authorships of the disputed papers. The text of each paper is available at `https://dlsun.github.io/pods/data/federalist/x.txt`, where `x` is the number of the paper (i.e., a number from 1 to 85). The name of the file indicates the number of the paper.

## Question 1

When analyzing an author's style, common words like "the" and "on" are actually more useful than rare words like "hostilities". That is because rare words typically signify context. Context is useful if you are trying to find documents about similar topics, but not so useful if you are trying to identify an author's style because different authors can write about the same topic. For example, both Dr. Seuss and Charles Dickens used rare words like "chimney" and "stockings" in _How the Grinch Stole Christmas_ and _A Christmas Carol_, respectively. But they used common words very differently: Dickens used the word "upon" over 100 times, while Dr. Seuss did not use "upon" even once.

Read in the Federalist Papers. Convert each one into a vector of term frequencies. In order to restrict to common words, include only the top 50 words across the corpus. (Because we are restricting to the most common words already, there is no reason to reweight them using TF-IDF.)

In [47]:
import pandas as pd
import requests
from collections import Counter

fed_dir = "https://dlsun.github.io/pods/data/"
docs_fed = pd.Series()

i = 1
for i in range(86):
  response = requests.get(fed_dir + "federalist/" + str(i) + '.txt', "r")
  docs_fed["federalist" + str(i)] = response.text

# docs_fed

words = (docs_fed.
         str.lower().
        #  str.split("/[^'’\p{L}\p{M}]+/").
        # ^'\w
         str.replace("[^'\w\s]", " ").
         str.split()
).apply(Counter)

words
# words.apply(Counter)
# words

# lst = []
# for word in words:
#   if word != 0:
#     lst = lst + word

# Counter(lst).most_common(50)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: FutureWarning: The default value of regex will change from True to False in a future version.


federalist0     {'doctype': 1, 'html': 5, 'head': 2, 'meta': 2...
federalist1     {'to': 72, 'the': 133, 'people': 6, 'of': 106,...
federalist2     {'to': 53, 'the': 107, 'people': 23, 'of': 83,...
federalist3     {'to': 56, 'the': 93, 'people': 8, 'of': 62, '...
federalist4     {'to': 51, 'the': 86, 'people': 8, 'of': 72, '...
                                      ...                        
federalist81    {'to': 163, 'the': 389, 'people': 1, 'of': 248...
federalist82    {'to': 83, 'the': 168, 'people': 1, 'of': 94, ...
federalist83    {'to': 219, 'the': 485, 'people': 3, 'of': 331...
federalist84    {'to': 140, 'the': 390, 'people': 11, 'of': 29...
federalist85    {'to': 115, 'the': 246, 'people': 7, 'of': 172...
Length: 86, dtype: object

In [48]:
tf = pd.DataFrame(list(words))
tf = tf.fillna(0)
tf

,doctype,html,head,meta,http,equiv,content,type,text,charset,utf,8,security,policy,default,src,'none',style,'unsafe,inline',img,data,connect,'self',title,page,not,found,middot,github,pages,css,media,screen,body,background,color,f1f1f1,margin,0,...,rail,excusing,insincerity,affectation,undermined,planted,prostration,asperities,disputants,rung,concealments,intemperances,excuse,aspersions,'t,constrained,bands,honestly,beware,dissemble,irrevocably,pamphlet,published,utter,deliberated,moulding,arranging,intents,vanishes,rare,abate,dint,demagogue,prodigy,completion,trembling,recommence,hume's,essays,128
0,1.0,5.0,2.0,2.0,3.0,2.0,4.0,2.0,6.0,1.0,1.0,1.0,1.0,1.0,1.0,6.0,1.0,4.0,1.0,1.0,8.0,3.0,1.0,1.0,4.0,1.0,3,2.0,1.0,4.0,3.0,1.0,2.0,7.0,3.0,1.0,5.0,1.0,8.0,13.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,14,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10,2.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,13,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,14,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
81,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,26,4.0,0.0,0.0,0.0,0.0,0.0,0.0,12.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
82,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,13,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
83,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,36,3.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
84,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,30,4.0,0.0,0.0,0.0,0.0,0.0,1.0,4.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Question 2
Make a visualization that summarizes the most common words used by Hamilton, Madison, and Jay.

In [49]:
word.most_common(50)

for w in word.most_common(50):
  w.plot.hist(bins=50)




AttributeError: ignored

## Question 3

For each of the documents with disputed authorships, find the 5 most similar documents with _known_ authorships, using cosine distance on the term frequencies. Use the authors of these 5 most similar documents to predict the author of each disputed document. (For example, if 3 of the 5 closest documents were written by Hamilton, 1 by Madison, and 1 by Jay, then we would predict that the disputed document was written by Hamilton.)

In [50]:
author_dir = "https://dlsun.github.io/pods/data/federalist/authorship.csv"
df_auth = pd.read_csv(author_dir)

df_auth

from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np

vec = TfidfVectorizer(norm=None) # Do not normalize.
vec.fit(docs_fed) # This determines the vocabulary.
tf_idf_sparse = vec.transform(docs_fed)
tf_idf_sparse

# Calculate the numerator.
a = tf_idf_sparse[0, :]
b = tf_idf_sparse[2, :]
dot = a.multiply(b).sum()

# Calculate the terms in the denominator.
a_len = np.sqrt(a.multiply(a).sum())
b_len = np.sqrt(b.multiply(b).sum())

# Cosine similarity is their ratio.
dot / (a_len * b_len)

# Calculate the numerators.
a = tf_idf_sparse[0, :]
B = tf_idf_sparse
dot = a.multiply(B).sum(axis=1)
dot

matrix([[23215.86770533],
        [ 1337.85889462],
        [ 1296.50147065],
        [ 1027.33133137],
        [ 1326.01479839],
        [  988.41773338],
        [ 1660.27060956],
        [ 1637.61777713],
        [ 1371.38973346],
        [ 1437.48545182],
        [ 2431.87363998],
        [ 1707.48418085],
        [ 1557.95944322],
        [  611.66244377],
        [ 1872.85414582],
        [ 2387.1730393 ],
        [ 1559.6490004 ],
        [ 1310.95128031],
        [ 1886.81735116],
        [ 1812.11843191],
        [ 1223.29823265],
        [ 1479.31725512],
        [ 2525.74138464],
        [ 1549.1490396 ],
        [ 1444.52709091],
        [ 1489.77112837],
        [ 1688.66303051],
        [ 1148.4576095 ],
        [ 1287.91635485],
        [ 1889.01575784],
        [ 1363.90263751],
        [ 1343.42365281],
        [ 1224.51190267],
        [ 1284.70985533],
        [ 1810.02161595],
        [ 1727.41851285],
        [ 2203.64456384],
        [ 2263.10233307],
        [ 24

## Submission Instructions

- Copy this notebook to your own Drive, if you have not already.
- Restart this notebook and run the cells from beginning to end. 
  - Go to Runtime > Restart and Run All.
- Rename this notebook by clicking on "DATA 301 Lab 5 - YOUR NAMES HERE" at the very top of this page. Replace "YOUR NAMES HERE" with the first and last names of you (and your partners, for Phase 2).
- Get the link to your notebook:
  - Click on "Share" at the top-right. 
  - Change the settings to "Anyone with the link can view". 
  - Copy the sharing link into Canvas.